In [ ]:
%load_ext autoreload
%autoreload 2

import locale
def getpreferredencoding(do_setlocale = True):
    return "UTF-8"
locale.getpreferredencoding = getpreferredencoding

!pip -q install datasets
!pip -q install transformers
!pip -q install peft
!pip -q install -U bitsandbytes
!pip -q install accelerate
!pip -q install rouge


import numpy as np
from datasets import load_dataset

from transformers import AutoTokenizer, AutoModelForCausalLM
from google.colab import drive
drive.mount('/content/drive')

#%cd /content/drive/MyDrive/CS6220 Folder #add direct access to the folder if you get an error in this cell
import os
os.chdir("/content/drive/MyDrive/CS6220 Folder")

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 521.2/521.2 kB 7.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 115.3/115.3 kB 10.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 9.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 174.7/174.7 kB 5.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 265.7/265.7 kB 24.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 92.6/92.6 MB 2.7 MB/s eta 0:00:00
Mounted at /content/drive


In [ ]:
device="cuda"
!pip install huggingface-cli
!huggingface-cli login

ERROR: Could not find a version that satisfies the requirement huggingface-cli (from versions: none)
ERROR: No matching distribution found for huggingface-cli
Token will not been saved to git credential helper. Pass `add_to_git_credential=True` if you want to set the git credential as well.
Token is valid (permission: read).
Your token has been saved to /root/.cache/huggingface/token
Login successful


In [ ]:
import numpy as np
from datasets import load_dataset

from transformers import (
    AutoTokenizer,
    AutoModelWithLMHead,
    AutoModelForCausalLM,
    BitsAndBytesConfig,
    GenerationConfig,
)
from nltk.translate.bleu_score import sentence_bleu
import os


from datasets import load_dataset

import torch
from torch.utils.data import Dataset

from peft import LoraConfig, prepare_model_for_kbit_training, get_peft_model

import pickle

import time
from tqdm import tqdm

import copy

import random

from bs4 import BeautifulSoup

## Aggregate

In [ ]:
model_name = 'codeparrot/codeparrot'

# quant_config = BitsAndBytesConfig(
#     load_in_4bit=True,
#     bnb_4bit_use_double_quant=True,
#     bnb_4bit_quant_type="nf4",
#     bnb_4bit_compute_dtype=torch.bfloat16,
# )

target_modules = ["lm_head", "c_proj"]
lora_config = LoraConfig(
    r=8,
    target_modules=target_modules,
    lora_alpha=32,
    lora_dropout=0.05,
    bias="none",
    task_type="CAUSAL_LM",
)

In [ ]:
SAVE_PATH = "./ronit_setup/checkpoints/global_model"
LOSSES_PATH = "./ronit_setup/logs/losses/global_model"
TIMES_PATH = "./ronit_setup/logs/times/global_model"

In [ ]:
def aggregate_weights(SAVE_PATH,
                      num_clients=2):

    weight_dict = {}

    for client_idx in range(num_clients):

        # Load client
        client_model = AutoModelForCausalLM.from_pretrained(
            SAVE_PATH + f"_{client_idx}", load_in_8bit = True, device_map={"": 0}
        )
        client_model = prepare_model_for_kbit_training(client_model)
        client_model = get_peft_model(client_model, lora_config).to(device)

        for k,v in client_model.state_dict().items():
            if isinstance(v, str):
                  weight_dict[k] = v
            else:
                if client_idx == 0:       # For the first client, we need to initialize the value.
                    weight_dict[k] = 0
                weight_dict[k] += v/torch.tensor(num_clients)

    return weight_dict

In [ ]:
# Make sure this config matches inside the aggregate weights function
global_model = AutoModelForCausalLM.from_pretrained(
    model_name, load_in_8bit = True, device_map={"": 0}
)
global_model.gradient_checkpointing_enable()
global_model = prepare_model_for_kbit_training(global_model)
global_model = get_peft_model(global_model, lora_config).to(device)

# Aggregate the weights by simply averaging
aggregated_weight_dict = aggregate_weights(SAVE_PATH)

global_model.load_state_dict(aggregated_weight_dict)

config.json:   0%|          | 0.00/927 [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/6.17G [00:00<?, ?B/s]

In [ ]:
# Save the aggregated model

global_model.save_pretrained(SAVE_PATH + "_aggregated_model")